# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Solution 1
##### Create a table session history with sessionID as partition key and itemInSession as clustering key. 
##### Attributes: SessionID, itemInSession, artist, song, length

In [10]:
query = """ CREATE TABLE IF NOT EXISTS session_history
            (sessionId int, 
             itemInSession int, 
             artist text, 
             song text, 
             length float, 
             PRIMARY KEY(sessionId, itemInSession)
             ) """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_history (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query = """ select artist, song, length 
            from session_history 
            where sessionID = 338 
            and itemInSession = 4 """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist: {}, Song: {}, Length of song: {}".format(row.artist, row.song, row.length))

Artist: Faithless, Song: Music Matters (Mark Knight Dub), Length of song: 495.30731201171875


### Solution 2
##### Create table user_history with userID and sessionID as composite partition key as combination of both would make row unique for paritioning and itemInSession as clustering key so the data would be sorted based in itemInSession as stated in requirement (song (sorted by itemInSession))
##### Attributes: userId, firstName, lastName, artist, song, itemInSession, sessionId

In [15]:
query = """ CREATE TABLE IF NOT EXISTS user_history 
            (userId int, 
             sessionId int, 
             itemInSession int,
             firstName text, 
             lastName text,  
             artist text, 
             song text,
             PRIMARY KEY((userId, sessionId), itemInSession)
             ) """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (userId, sessionId, itemInSession, firstName, lastName,  artist, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [17]:
query = """ select firstName, lastName, artist, song 
            from user_history 
            where userID = 10 
            and sessionId = 182 """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist: {}, Song: {}, User: {} {}".format(row.artist, row.song, row.firstname, row.lastname))

Artist: Down To The Bone, Song: Keep On Keepin' On, User: Sylvie Cruz
Artist: Three Drives, Song: Greece 2000, User: Sylvie Cruz
Artist: Sebastien Tellier, Song: Kilometer, User: Sylvie Cruz
Artist: Lonnie Gordon, Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), User: Sylvie Cruz


### Solution 3
##### Create table listen_history with song as partition key and userID clustering key. List of users are required that listened to specific song. The paritioning on song with userID as clustering key will privide fasted results.
##### Attributes: userId, firstName, lastName, song

In [18]:
query = """ CREATE TABLE IF NOT EXISTS listen_history
            (song text,
             userID int, 
             firstName text, 
             lastName text,  
             PRIMARY KEY (song, userId)
             )"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO listen_history (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [21]:
query = """select firstname, lastname
           from listen_history 
           where song='All Hands Against His Own'
           """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("User: {} {}".format(row.firstname, row.lastname))

User: Jacqueline Lynch
User: Tegan Levine
User: Sara Johnson


### Drop the tables before closing out the sessions

In [22]:
query = "drop table session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table listen_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()